In [4]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count


In [5]:
complete_df=pd.read_csv("fra_cleaned.csv", sep=';' , encoding='latin-1')
complete_df.head()

,url,Perfume,Brand,Country,Gender,Rating Value,Rating Count,Year,Top,Middle,Base,Perfumer1,Perfumer2,mainaccord1,mainaccord2,mainaccord3,mainaccord4,"mainaccord5,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
0,https://www.fragrantica.com/perfume/xerjoff/ac...,accento-overdose-pride-edition,xerjoff,Italy,unisex,"1,42",201,2022.0,"fruity notes, aldehydes, green notes","bulgarian rose, egyptian jasmine, lily-of-the-...","eucalyptus, pine",unknown,NaN,rose,woody,fruity,aromatic,"floral,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,..."
1,https://www.fragrantica.com/perfume/jean-paul-...,classique-pride-2024,jean-paul-gaultier,France,women,"1,86",70,2024.0,"yuzu, citruses","orange blossom, neroli","musk, blonde woods",unknown,NaN,citrus,white floral,sweet,fresh,"musky,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,..."
2,https://www.fragrantica.com/perfume/jean-paul-...,classique-pride-2023,jean-paul-gaultier,France,unisex,"1,91",285,2023.0,"blood orange, yuzu","neroli, orange blossom","musk, white woods",natalie gracia-cetto,quentin bisch,citrus,white floral,sweet,fresh spicy,"musky,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,..."
3,https://www.fragrantica.com/perfume/bruno-bana...,pride-edition-man,bruno-banani,Germany,men,"1,92",59,2019.0,"guarana, grapefruit, red apple","walnut, lavender, guava","vetiver, benzoin, amber",unknown,NaN,fruity,nutty,woody,tropical,",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,..."
4,https://www.fragrantica.com/perfume/jean-paul-...,le-male-pride-collector,jean-paul-gaultier,France,men,"1,93",632,2020.0,"mint, lavender, cardamom, artemisia, bergamot","caraway, cinnamon, orange blossom","vanilla, sandalwood, amber, cedar, tonka bean",francis kurkdjian,NaN,aromatic,warm spicy,fresh spicy,cinnamon,"vanilla,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,..."


In [6]:
complete_df.rename(
    columns={
        "mainaccord5,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,": "mainaccord5"
    },
    inplace=True,
)
complete_df["mainaccord5"] = complete_df["mainaccord5"].str.replace(",", "")
complete_df.head()

,url,Perfume,Brand,Country,Gender,Rating Value,Rating Count,Year,Top,Middle,Base,Perfumer1,Perfumer2,mainaccord1,mainaccord2,mainaccord3,mainaccord4,mainaccord5
0,https://www.fragrantica.com/perfume/xerjoff/ac...,accento-overdose-pride-edition,xerjoff,Italy,unisex,"1,42",201,2022.0,"fruity notes, aldehydes, green notes","bulgarian rose, egyptian jasmine, lily-of-the-...","eucalyptus, pine",unknown,NaN,rose,woody,fruity,aromatic,floral
1,https://www.fragrantica.com/perfume/jean-paul-...,classique-pride-2024,jean-paul-gaultier,France,women,"1,86",70,2024.0,"yuzu, citruses","orange blossom, neroli","musk, blonde woods",unknown,NaN,citrus,white floral,sweet,fresh,musky
2,https://www.fragrantica.com/perfume/jean-paul-...,classique-pride-2023,jean-paul-gaultier,France,unisex,"1,91",285,2023.0,"blood orange, yuzu","neroli, orange blossom","musk, white woods",natalie gracia-cetto,quentin bisch,citrus,white floral,sweet,fresh spicy,musky
3,https://www.fragrantica.com/perfume/bruno-bana...,pride-edition-man,bruno-banani,Germany,men,"1,92",59,2019.0,"guarana, grapefruit, red apple","walnut, lavender, guava","vetiver, benzoin, amber",unknown,NaN,fruity,nutty,woody,tropical,
4,https://www.fragrantica.com/perfume/jean-paul-...,le-male-pride-collector,jean-paul-gaultier,France,men,"1,93",632,2020.0,"mint, lavender, cardamom, artemisia, bergamot","caraway, cinnamon, orange blossom","vanilla, sandalwood, amber, cedar, tonka bean",francis kurkdjian,NaN,aromatic,warm spicy,fresh spicy,cinnamon,vanilla


#Column Details

URL: Same as the raw dataset, the link to the fragrance's page.

Perfume: The name of the fragrance.

Brand: Standardized names of the fragrance brands.

Country: Standardized country names for easier analysis.

Gender: Cleaned and unified categories for the fragrance’s marketed gender (e.g., men, women, unisex).

Rating Value: Cleaned numeric ratings (decimal format).

Rating Count: Standardized rating counts.

Year: The launch year in a cleaned and unified format.

Top Notes: Parsed and standardized scent notes.

Middle Notes: Parsed and standardized middle notes.

Base Notes: Parsed and standardized base notes.

Perfumer1, Perfumer2: Standardized and separated names of the perfumers.

Main Accord 1-5: Cleaned and separated into distinct scent families.

In [17]:
complete_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24063 entries, 0 to 24062
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url           24063 non-null  object 
 1   Perfume       24063 non-null  object 
 2   Brand         24063 non-null  object 
 3   Country       24063 non-null  object 
 4   Gender        24063 non-null  object 
 5   Rating Value  24063 non-null  object 
 6   Rating Count  24063 non-null  int64  
 7   Year          22026 non-null  float64
 8   Top           24063 non-null  object 
 9   Middle        24063 non-null  object 
 10  Base          24063 non-null  object 
 11  Perfumer1     24063 non-null  object 
 12  Perfumer2     1336 non-null   object 
 13  mainaccord1   24063 non-null  object 
 14  mainaccord2   24050 non-null  object 
 15  mainaccord3   23949 non-null  object 
 16  mainaccord4   23675 non-null  object 
 17  mainaccord5   24063 non-null  object 
dtypes: float64(1), int64(1), o

In [18]:
complete_df.nunique()

url             24063
Perfume         22840
Brand            1060
Country            54
Gender              3
Rating Value      221
Rating Count     2721
Year              145
Top             18771
Middle          19690
Base            16393
Perfumer1         869
Perfumer2         259
mainaccord1        66
mainaccord2        71
mainaccord3        72
mainaccord4        69
mainaccord5        78
dtype: int64

In [19]:
print(complete_df.isna().sum())

url                 0
Perfume             0
Brand               0
Country             0
Gender              0
Rating Value        0
Rating Count        0
Year             2037
Top                 0
Middle              0
Base                0
Perfumer1           0
Perfumer2       22727
mainaccord1         0
mainaccord2        13
mainaccord3       114
mainaccord4       388
mainaccord5         0
dtype: int64


In [9]:
columns=list(complete_df.columns)
print(columns)

['url', 'Perfume', 'Brand', 'Country', 'Gender', 'Rating Value', 'Rating Count', 'Year', 'Top', 'Middle', 'Base', 'Perfumer1', 'Perfumer2', 'mainaccord1', 'mainaccord2', 'mainaccord3', 'mainaccord4', 'mainaccord5']


In [28]:
complete_df['Rating Value'].head(24000)

0        1,42
1        1,86
2        1,91
3        1,92
4        1,93
         ... 
23995    4,66
23996    4,67
23997    4,67
23998    4,67
23999    4,67
Name: Rating Value, Length: 24000, dtype: object

# Spark code


In [20]:
spark = SparkSession.builder \
    .appName("PerfumeRecommendation") \
    .config("spark.executor.extraJavaOptions", "-Djava.security.manager=allow") \
    .config("spark.driver.extraJavaOptions", "-Djava.security.manager=allow") \
    .getOrCreate()


In [21]:
df = spark.createDataFrame(complete_df, columns)

# Count repetitions in the 'Perfume' column
repeating_perfume = df.groupBy("Perfume") \
                      .agg(count("Perfume").alias("Count")) \
                      .filter(col("Count") > 1)

# Join with original DataFrame to get corresponding 'url' values
result = repeating_perfume.join(df, on="Perfume", how="inner")

In [22]:
result.show()

+-------------+-----+--------------------+--------------------+-------+------+------------+------------+------+--------------------+--------------------+--------------------+--------------------+---------------+------------+------------+------------+------------+-----------+
|      Perfume|Count|                 url|               Brand|Country|Gender|Rating Value|Rating Count|  Year|                 Top|              Middle|                Base|           Perfumer1|      Perfumer2| mainaccord1| mainaccord2| mainaccord3| mainaccord4|mainaccord5|
+-------------+-----+--------------------+--------------------+-------+------+------------+------------+------+--------------------+--------------------+--------------------+--------------------+---------------+------------+------------+------------+------------+-----------+
|         hope|    2|https://www.fragr...|         orlov-paris| France| women|         3,5|          42|2020.0|violet leaf, pink...|peach, osmanthus,...|musk, cedar, patc..

In [23]:
result.printSchema()

root
 |-- Perfume: string (nullable = true)
 |-- Count: long (nullable = false)
 |-- url: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Rating Value: string (nullable = true)
 |-- Rating Count: long (nullable = true)
 |-- Year: double (nullable = true)
 |-- Top: string (nullable = true)
 |-- Middle: string (nullable = true)
 |-- Base: string (nullable = true)
 |-- Perfumer1: string (nullable = true)
 |-- Perfumer2: string (nullable = true)
 |-- mainaccord1: string (nullable = true)
 |-- mainaccord2: string (nullable = true)
 |-- mainaccord3: string (nullable = true)
 |-- mainaccord4: string (nullable = true)
 |-- mainaccord5: string (nullable = true)



In [24]:
result.count()

2043